Read in `inputs.yml` and produce a GraphViz representation of the tables

In [1]:
import os
import yaml

import cea.scripts
import cea.interfaces.dashboard.inputs

from jinja2 import Template

import cea.inputlocator
import cea.config

config = cea.config.Configuration()
locator = cea.inputlocator.InputLocator(scenario=config.scenario)
schemas = cea.scripts.schemas()

OUTPUT_PATH = os.path.join(os.path.dirname(cea.config.__file__), "..", "docs", "tables_diagram.gv")

In [2]:
inputs_yml = os.path.abspath(os.path.join(os.path.dirname(cea.interfaces.dashboard.inputs.__file__), "inputs.yml"))

with open(inputs_yml, 'r') as fp:
    inputs = yaml.load(fp)

inputs

{'air-conditioning-systems': {'fields': [{'name': 'Name', 'type': 'str'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'COOLING'}},
    'name': 'type_cs',
    'type': 'choice'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'HEATING'}},
    'name': 'type_hs',
    'type': 'choice'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'HOT_WATER'}},
    'name': 'type_dhw',
    'type': 'choice'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'CONTROLLER'}},
    'name': 'type_ctrl',
    'type': 'choice'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'VENTILATION'}},
    'name': 'type_vent',
    'type': 'ch

In [3]:
table_template = """
{{locator}} [shape=none, margin=0, label=<
    <table border="0" cellborder="1" cellspacing="0" cellpadding="4" >
        <tr><td bgcolor="lightblue">{{table}}</td></tr>
        {% for field in fields %}<tr><td port="{{field.name}}" align="left">{{field.type}}: {{field.name}}</td></tr>
        {% endfor %}
    </table>>];
"""

In [4]:
table_defs = []

for table in inputs.keys():
    lm = inputs[table]["location"]
    fields = inputs[table]["fields"]
    table_defs.append(Template(table_template).render(locator=lm, table=table, fields=fields))

In [5]:
print "\n".join(table_defs)


get_building_air_conditioning [shape=none, margin=0, label=<
    <table border="0" cellborder="1" cellspacing="0" cellpadding="4" >
        <tr><td bgcolor="lightblue">air-conditioning-systems</td></tr>
        <tr><td port="Name" align="left">str: Name</td></tr>
        <tr><td port="type_cs" align="left">choice: type_cs</td></tr>
        <tr><td port="type_hs" align="left">choice: type_hs</td></tr>
        <tr><td port="type_dhw" align="left">choice: type_dhw</td></tr>
        <tr><td port="type_ctrl" align="left">choice: type_ctrl</td></tr>
        <tr><td port="type_vent" align="left">choice: type_vent</td></tr>
        <tr><td port="heat_starts" align="left">date: heat_starts</td></tr>
        <tr><td port="heat_ends" align="left">date: heat_ends</td></tr>
        <tr><td port="cool_starts" align="left">date: cool_starts</td></tr>
        <tr><td port="cool_ends" align="left">date: cool_ends</td></tr>
        
    </table>>];

get_zone_geometry [shape=none, margin=0, label=<
    

In [6]:
databases = set()  # (lm, sheet)
connections = set()  # con

for ltable in inputs:
    lm = inputs[ltable]["location"]
    for field in inputs[ltable]["fields"]:
        if field["type"] == "choice":
            lfield = field["name"]
            rtable = field["choice_properties"]["lookup"]["path"]
            sheet = field["choice_properties"]["lookup"]["sheet"]
            rfield = field["choice_properties"]["lookup"]["column"]
            databases.add((rtable, sheet))
            connections.add((lm, lfield, rtable, sheet, rfield))

databases

{('get_database_air_conditioning_systems', 'CONTROLLER'),
 ('get_database_air_conditioning_systems', 'COOLING'),
 ('get_database_air_conditioning_systems', 'HEATING'),
 ('get_database_air_conditioning_systems', 'HOT_WATER'),
 ('get_database_air_conditioning_systems', 'VENTILATION'),
 ('get_database_construction_standards', 'STANDARD_DEFINITION'),
 ('get_database_envelope_systems', 'CONSTRUCTION'),
 ('get_database_envelope_systems', 'FLOOR'),
 ('get_database_envelope_systems', 'ROOF'),
 ('get_database_envelope_systems', 'SHADING'),
 ('get_database_envelope_systems', 'TIGHTNESS'),
 ('get_database_envelope_systems', 'WALL'),
 ('get_database_envelope_systems', 'WINDOW'),
 ('get_database_supply_assemblies', 'COOLING'),
 ('get_database_supply_assemblies', 'ELECTRICITY'),
 ('get_database_supply_assemblies', 'HEATING'),
 ('get_database_supply_assemblies', 'HOT_WATER'),
 ('get_database_use_types_properties', 'INTERNAL_LOADS')}

In [7]:
connections

{('get_building_air_conditioning',
  'type_cs',
  'get_database_air_conditioning_systems',
  'COOLING',
  'code'),
 ('get_building_air_conditioning',
  'type_ctrl',
  'get_database_air_conditioning_systems',
  'CONTROLLER',
  'code'),
 ('get_building_air_conditioning',
  'type_dhw',
  'get_database_air_conditioning_systems',
  'HOT_WATER',
  'code'),
 ('get_building_air_conditioning',
  'type_hs',
  'get_database_air_conditioning_systems',
  'HEATING',
  'code'),
 ('get_building_air_conditioning',
  'type_vent',
  'get_database_air_conditioning_systems',
  'VENTILATION',
  'code'),
 ('get_building_architecture',
  'type_base',
  'get_database_envelope_systems',
  'FLOOR',
  'code'),
 ('get_building_architecture',
  'type_cons',
  'get_database_envelope_systems',
  'CONSTRUCTION',
  'code'),
 ('get_building_architecture',
  'type_floor',
  'get_database_envelope_systems',
  'FLOOR',
  'code'),
 ('get_building_architecture',
  'type_leak',
  'get_database_envelope_systems',
  'TIGHTNESS'

In [8]:
db_template = """
{{locator}}_{{sheet}} [shape=none, margin=0, label=<
    <table border="0" cellborder="1" cellspacing="0" cellpadding="4" >
        <tr><td bgcolor="lightgrey">{{table}}</td></tr>
        {% for field in fields %}<tr><td port="{{field.name}}" align="left">{{field.type}}: {{field.name}}</td></tr>
        {% endfor %}
    </table>>];
"""

db_defs = []

for lm, sheet in databases:
    xlsx = os.path.basename(getattr(locator, lm)())
    table = "{file}:{sheet}".format(file=xlsx, sheet=sheet)
    fields = []
    for fn in schemas[lm]["schema"][sheet].keys():
        name = fn.replace("&", "")
        fields.append({"name": name, "type": schemas[lm]["schema"][sheet][fn]["types_found"][0]})
    db_defs.append(Template(db_template).render(locator=lm, sheet=sheet, table=table, fields=fields))
    
print db_defs[0]


get_database_envelope_systems_ROOF [shape=none, margin=0, label=<
    <table border="0" cellborder="1" cellspacing="0" cellpadding="4" >
        <tr><td bgcolor="lightgrey">ENVELOPE.xls:ROOF</td></tr>
        <tr><td port="e_roof" align="left">float: e_roof</td></tr>
        <tr><td port="code" align="left">string: code</td></tr>
        <tr><td port="Description" align="left">string: Description</td></tr>
        <tr><td port="GHG_ROOF_kgCO2m2" align="left">float: GHG_ROOF_kgCO2m2</td></tr>
        <tr><td port="a_roof" align="left">float: a_roof</td></tr>
        <tr><td port="r_roof" align="left">float: r_roof</td></tr>
        <tr><td port="U_roof" align="left">float: U_roof</td></tr>
        
    </table>>];


In [9]:
con_defs = []

for ltable, lfield, rtable, sheet, rfield in connections:
    con_defs.append('{ltable}:"{lfield}" -> {rtable}_{sheet}:{rfield};'.format(ltable=ltable, lfield=lfield, rtable=rtable, sheet=sheet, rfield=rfield))

con_defs

['get_building_air_conditioning:"type_cs" -> get_database_air_conditioning_systems_COOLING:code;',
 'get_building_air_conditioning:"type_ctrl" -> get_database_air_conditioning_systems_CONTROLLER:code;',
 'get_building_typology:"STANDARD" -> get_database_construction_standards_STANDARD_DEFINITION:STANDARD;',
 'get_building_typology:"2ND_USE" -> get_database_use_types_properties_INTERNAL_LOADS:code;',
 'get_building_air_conditioning:"type_dhw" -> get_database_air_conditioning_systems_HOT_WATER:code;',
 'get_building_air_conditioning:"type_vent" -> get_database_air_conditioning_systems_VENTILATION:code;',
 'get_building_architecture:"type_leak" -> get_database_envelope_systems_TIGHTNESS:code;',
 'get_building_architecture:"type_cons" -> get_database_envelope_systems_CONSTRUCTION:code;',
 'get_building_architecture:"type_base" -> get_database_envelope_systems_FLOOR:code;',
 'get_building_supply:"type_dhw" -> get_database_supply_assemblies_HOT_WATER:code;',
 'get_building_air_conditioning:"

In [10]:
diagram_template = """
/* this diagram was generated by notebooks/2020.02.25-create-tables-entity-relationship-diagram.ipynb 
   you can preview it in VSCode with the "Graphviz Preview" extension.
*/
digraph tables_diagram {

    graph [pad="0.5", nodesep="0.5", ranksep="2"];
    node [shape=plain];
    rankdir=LR;
    
{% for table in tables %}
   {{table}}
{% endfor %}

{% for db in dbs %}
    {{db}}
{% endfor %}

{% for con in cons %}
    {{con}}
{% endfor %}
}
"""


with open(OUTPUT_PATH, 'w') as fp:
    fp.write(Template(diagram_template).render(tables=table_defs, dbs=db_defs, cons=con_defs))